# Combine product concentrations from GC with enzymatic assays and OD-pH-pressure

## Imports and setup

In [52]:
import os
from pathlib import Path
import webbrowser
from functools import reduce
import random
import operator
import pandas as pd
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
import plotly.offline as py
import plotly.graph_objs as go
import plotly_express as px
import numpy as np
import sympy as sp
import qgrid as qg
import plotly.io as pio

from utils.plot_utils import scatterplot_from_df, boxplot_from_df 

idx = pd.IndexSlice

sp.init_printing()
py.init_notebook_mode(connected=True)
qg.set_defaults(show_toolbar=True)
qg.enable()

## Input files and definitions

In [2]:
sample_template = pd.DataFrame(
    [
        (sample, bottle) 
        for sample in range(1, 12, 1) 
        for bottle in ["A", "B", "C", "D", "E", "F", "G", "H"]
    ],
    columns=("Sample", "Bottle")
).set_index(["Sample", "Bottle"])

sample_template

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [55]:
sample_times_path = Path("./data/sample-times_CO.csv")
gc_conc_path = Path("./intermediate/GC/CO/concentrations.csv")
od_ph_path = Path("./data/growth_CO.csv")
eth_ac_assay_path = Path("./data/Enzymatic_Assays/ethanol_acetate_assay_results_CO.csv")
fru_assay_path = Path("./data/Enzymatic_Assays/fructose_assay_results_CO.csv")
form_assay_path = Path("./data/Enzymatic_Assays/formate_assay_results_CO.csv")

intermediate_path = Path("./intermediate")
results_path = Path("./results/CO")

In [26]:
def query_group(g):
    return (
        ((g.index.get_level_values("Bottle_Set") == "A-D") & 
         g.index.get_level_values("Bottle").isin(["A", "B", "C", "D"])) | 
        ((g.index.get_level_values("Bottle_Set") == "E-H") & 
         g.index.get_level_values("Bottle").isin(["E", "F", "G", "H"]))
    ) 

exp_from_bottle_col = lambda bottle: (
    np.where(
        bottle.isin(["A", "B", "C", "D"]), 
        "control", 
        np.where(
            bottle.isin(["E", "F", "G", "H"]), 
            "pressure", 
            float("NaN")
        )
    )
)

sample_times = sample_template.join(
    pd.read_csv(
        sample_times_path
    ).rename(
        {
            "Bottle Set": "Bottle_Set"
        },
        axis="columns"
    ).set_index(
        ["Sample", "Bottle_Set"]
    )
).groupby(
    level=["Sample", "Bottle"]
).agg({
    "Time": lambda group: group.loc[query_group(group)].iloc[0]
}).assign(
    Condition=lambda df: exp_from_bottle_col(df.index.get_level_values("Bottle"))
)

sample_times

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [53]:
psi_to_bar = 0.0689476

qualities = {
    "OD 600": {
        "Quantity": "Optical Extinction",
        "Unit": "1"
    },
    "pH": {
        "Quantity": "pH",
        "Unit": "1"
    },
    "Pressure after": {
        "Quantity": "Absolute Pressure",
        "Unit": "bar"
    }
}

def convert_psi_to_bar(df):
    mask_all = (df.Quality == "pressure_after")
    mask_high = (df.Value > 10) 
    new_df = df.copy()
    new_df.loc[mask_all & mask_high, "Value"] *= psi_to_bar
    new_df.loc[mask_all, "Value"] += 1.01325
    return new_df

growth_data = sample_times.join(
    pd.read_csv(od_ph_path, index_col=["Sample", "Bottle"]).loc[:,["OD_600", "pH", "pressure_after"]]
).rename(
    {
        "pressure_after": "Pressure after",
        "OD_600": "OD 600"
    },
    axis="columns"
).assign(
    Method="Manual"
).reset_index().melt(
    id_vars=["Sample", "Bottle", "Time", "Method", "Condition"],
    value_vars=["OD 600", "pH", "Pressure after"],
    var_name="Quality",
    value_name="Value"
).pipe(
    convert_psi_to_bar
).assign(
    Quantity=lambda df: df.Quality.map(lambda q: qualities[q]["Quantity"]),
    Unit=lambda df: df.Quality.map(lambda q: qualities[q]["Unit"]),
    StDev=0,
    Points=1
).set_index(
    ["Sample", "Bottle", "Quality"]
)

growth_data

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [28]:
qualities = {
    "Acetate": {
        "Quantity": "Concentration",
        "Unit": "g/L"
    },
    "EtOH": {
        "Quantity": "Concentration",
        "Unit": "g/L"
    }
}

gc_conc = sample_times.join(
    pd.read_csv(gc_conc_path, index_col=["Sample", "Bottle"])
).assign(
    Method="GC"
).reset_index().melt(
    id_vars=["Sample", "Bottle", "Time", "Method", "Condition"],
    value_vars=["Acetate", "EtOH"],
    var_name="Quality",
    value_name="Value"
).assign(
    Quantity=lambda df: df.Quality.map(lambda q: qualities[q]["Quantity"]),
    Unit=lambda df: df.Quality.map(lambda q: qualities[q]["Unit"]),
    StDev=0,
    Points=1
).set_index(["Sample", "Bottle", "Quality"])

gc_conc

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [30]:
assay_data = sample_times.join(
    [
        pd.read_csv(eth_ac_assay_path, index_col=["Sample", "Bottle"]),
        pd.read_csv(fru_assay_path, index_col=["Sample", "Bottle"]),
        pd.read_csv(form_assay_path, index_col=["Sample", "Bottle"]).rename(
            {
                "Formate": "Formic Acid"
            },
            axis="columns"
        ).reindex(
            ["Formic Acid"],
            axis="columns"
        )
    ]
)

assay_data

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [31]:
qualities = {
    "Acetate": {
        "Quantity": "Concentration",
        "Unit": "g/L"
    },
    "EtOH": {
        "Quantity": "Concentration",
        "Unit": "g/L"
    },
    "Fructose": {
        "Quantity": "Concentration",
        "Unit": "g/L"
    },
    "Formic Acid": {
        "Quantity": "Concentration",
        "Unit": "g/L"
    }
}

assay_conc = assay_data.loc[
    :, 
    ["Time", "Condition", "Acetate", "Ethanol", "Fructose", "Formic Acid"]
].rename(
    {
        "Ethanol": "EtOH"
    },
    axis="columns"
).assign(
    Method="Enzymatic_Assay"
).reset_index().melt(
    id_vars=["Sample", "Bottle", "Time", "Method", "Condition"],
    value_vars=["Acetate", "EtOH", "Fructose", "Formic Acid"],
    var_name="Quality",
    value_name="Value"
).assign(
    Quantity=lambda df: df.Quality.map(lambda q: qualities[q]["Quantity"]),
    Unit=lambda df: df.Quality.map(lambda q: qualities[q]["Unit"]),
    StDev=0,
    Points=1
).dropna(
).set_index(["Sample", "Bottle", "Quality"])

assay_conc

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [32]:
gc_vs_assays = pd.concat(
    [
        assay_conc[assay_conc.index.isin(gc_conc.index)],
        gc_conc.loc[gc_conc.index.isin(assay_conc.index)]
    ]
)

gc_vs_assays

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

## Some Defs for plotting

In [54]:
units_of_choice = {
    "Time": "h",
    "Concentration": "g/L",
    "Optical Extinction": "1",
    "pH": "1",
    "Absolute Pressure": "bar",
    "Relative Expression": "1",
    "Comparative Expression": "1"
}

quality_color_map = {
    "Acetate": "rgb(150,0,0)",
    "EtOH": "rgb(0,50,150)",
    "Fructose": "rgb(0,150,250)",
    "Formic Acid": "rgb(30,200,0)",
    "OD 600": "rgb(0,100,50)",
    "pH": "rgb(150,0,150)",
    "Pressure after": "rgb(0,0,50)",
    "aor": "rgb(250, 0, 150)",
    "pta": "rgb(200, 110, 0)",
    "fdh": "rgb(0, 150, 30)",
    "codh": "rgb(0, 30, 150)"
}

bottle_color_map = {
    "A": "rgb(200,0,0)",
    "B": "rgb(0,75,200)",
    "C": "rgb(0,200,255)",
    "D": "rgb(0,150,90)",
    "E": "rgb(240,50,240)",
    "F": "rgb(255, 200, 0)",
    "G": "rgb(255, 135, 0)",
    "H": "rgb(0, 200, 20)",
}

## Correlation between GC and Assays for Acetate (multi-scatter)

In [56]:
fig = px.scatter(
    gc_vs_assays.reset_index(
    ).pipe(
        lambda df: df[(df.Quality == "Acetate") & (df.Unit == "g/L")]
    ), 
    x="Time",
    y="Value",
    color="Method",
    facet_col="Bottle",
    template="plotly_white",
    labels={
        "Time": "Time [h]",
        "Value": "Concentration [g/L]",
        "Quality": "Compound"
    }
)

pio.write_image(fig, str(results_path / 'assay-gc-correlation_ac.svg'))
fig

## Correlation between GC and Assays for Ethanol (multi-scatter)

In [57]:
fig = px.scatter(
    gc_vs_assays.reset_index(
    ).pipe(
        lambda df: df[(df.Quality == "EtOH") & (df.Unit == "g/L")]
    ), 
    x="Time",
    y="Value",
    color="Method",
    facet_col="Bottle",
    template="plotly_white",
    labels={
        "Time": "Time [h]",
        "Value": "Concentration [g/L]",
        "Quality": "Compound"
    }
)

pio.write_image(fig, str(results_path / 'assay-gc-correlation_eth.svg'))
fig

In [62]:
main_data = pd.concat(
    [
        gc_conc,
        growth_data,
        assay_conc.loc[idx[:, :, ["Fructose", "Formic Acid"]], :]
    ],
    sort=False
)

main_data

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [63]:
mean_data = main_data.reset_index().groupby(
    by=["Sample", "Method", "Quality", "Condition", "Quantity", "Unit"]
).agg(
    dict(
        Value=["mean", "std", "count"],
        Time="first"
    )
)

mean_data.columns = mean_data.columns.droplevel()
mean_data = mean_data.rename(
    {
        "mean": "Value",
        "std": "StDev",
        "count": "Points",
        "first": "Time"
    }, 
    axis="columns"
).reset_index().set_index(["Sample", "Quality"])

mean_data

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [60]:
def gen_plotly_layout(axis_map, measures):
    return {
        "xaxis": dict(
            domain=[0, (0.92 if len(measures) > 1 else 1)],
            title="Time"+" [%s]"%(units_of_choice["Time"]),
        ),
        **{
            "yaxis"+(str(ops["index"]) if ops["index"] != 1 else ""): dict(
                title=quant+" [%s]"%(units_of_choice[(quant)]),
                anchor="x",
                side=ops["position"],
                overlaying=("y" if ops["index"] != 1 else None)
            ) for quant, ops in axis_map.items()
        }
    }

def plot_measurements(df, measures, axis_map):
    def create_scatter(m, c, yaxis, bottle=None):
        data = df.reset_index(
        ).pipe(
            lambda df: df.loc[
                (df.Condition == c) &
                (df.Quality == m[0]) &
                (df.Quantity == m[1])
            ]
        ).pipe(
            lambda df: df.loc[df.Bottle == bottle] if bottle else df
        )
        
        return go.Scatter(
            x=data.Time,
            y=data.Value,
            error_y=dict(
                type='data',
                array=data.StDev,
                visible=(True if data.StDev.sum() > 0 else False)
            ),
            mode="lines+markers",
            name="%s%s%s"%(
                "%s: "%(m[0]) if not bottle else "", 
                "5 atm" if c == "pressure" else "1 atm", 
                " (%s)"%(bottle) if bottle else ""
            ),
            marker=dict(
                color=(bottle_color_map[bottle] if bottle else quality_color_map[m[0]])
            ),
            line=dict(
                dash=("dash" if c == "control" else None)
            ),
            opacity=0.8,
            yaxis=yaxis
        )
    
    nested_data = [
        (
            [
                create_scatter(
                    m, 
                    c,
                    "y"+str(axis_map[m[1]]["index"])
                )
            ]  
                if mode == "mean" else
            [
                create_scatter(
                    m, 
                    c, 
                    "y"+str(axis_map[m[1]]["index"]),
                    bottle=b
                ) 
                for b, inner_df in df.reset_index().groupby(by="Bottle") 
            ]
        ) 
        for m, mode in measures.items() 
        for c in ["control", "pressure"]
    ]
    
    return dict(
        data=[d for sub in nested_data for d in sub],
        layout=gen_plotly_layout(axis_map, measures)
    )

## Pressure scatter

In [61]:
axis_map = {
    "Absolute Pressure": {
        "index": 1,
        "position": "left"
    }
}

fig1 = go.FigureWidget(plot_measurements(
    main_data.loc[main_data.Condition == "control"],
    {
        ("pressure_after", "Absolute Pressure"): "all"
    },
    axis_map
))

fig2 = go.FigureWidget(plot_measurements(
    main_data.loc[main_data.Condition == "pressure"],
    {
        ("pressure_after", "Absolute Pressure"): "all"
    },
    axis_map
))

pio.write_image(fig1, str(results_path / 'pressure_control.svg'))
fig1

FigureWidget({
    'data': [{'error_y': {'array': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64),
     …

In [64]:
pio.write_image(fig2, str(results_path / 'pressure_pressure.svg'))
fig2

FigureWidget({
    'data': [{'error_y': {'array': array([], dtype=int64), 'type': 'data', 'visible': False},
 …

## Fructose scatter

In [65]:
axis_map = {
    "Concentration": {
        "index": 1,
        "position": "left"
    }
}

fig = go.FigureWidget(
    plot_measurements(
        main_data, 
        {
            ("Fructose", "Concentration"): "all"
        }, 
        axis_map
    )
)

pio.write_image(fig, str(results_path / 'fructose.svg'))
fig

FigureWidget({
    'data': [{'error_y': {'array': array([0, 0, 0, 0], dtype=int64), 'type': 'data', 'visible':…

## OD_600 scatter

In [67]:
axis_map = {
    "Optical Extinction": {
        "index": 1,
        "position": "left"
    }
}

fig = go.FigureWidget(
    plot_measurements(
        main_data, 
        {
            ("OD 600", "Optical Extinction"): "all"
        }, 
        axis_map
    )
)

pio.write_image(fig, str(results_path / 'od-600.svg'))
fig

FigureWidget({
    'data': [{'error_y': {'array': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64),
     …

## pH scatter

In [68]:
axis_map = {
    "pH": {
        "index": 1,
        "position": "left"
    }
}

fig = go.FigureWidget(
    plot_measurements(
        main_data, 
        {
            ("pH", "pH"): "all"
        }, 
        axis_map
    )
)

pio.write_image(fig, str(results_path / 'pH.svg'))
fig

FigureWidget({
    'data': [{'error_y': {'array': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64),
     …

## Acetate scatter

In [69]:
axis_map = {
    "Concentration": {
        "index": 1,
        "position": "left"
    }
}

fig = go.FigureWidget(
    plot_measurements(
        main_data, 
        {
            ("Acetate", "Concentration"): "all"
        }, 
        axis_map
    )
)

pio.write_image(fig, str(results_path / 'acetate.svg'))
fig

FigureWidget({
    'data': [{'error_y': {'array': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64),
     …

## Ethanol scatter

In [70]:
axis_map = {
    "Concentration": {
        "index": 1,
        "position": "left"
    }
}

fig = go.FigureWidget(
    plot_measurements(
        main_data, 
        {
            ("EtOH", "Concentration"): "all"
        }, 
        axis_map
    )
)

pio.write_image(fig, str(results_path / 'ethanol.svg'))
fig

FigureWidget({
    'data': [{'error_y': {'array': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64),
     …

## Formic Acid scatter

In [71]:
axis_map = {
    "Concentration": {
        "index": 1,
        "position": "left"
    }
}

figure = plot_measurements(
    main_data, 
    {
        ("Formic Acid", "Concentration"): "all"
    }, 
    axis_map
)

figure['data'] = [
    *figure['data'], 
    go.Scatter({
        'mode': 'lines',
        'x': [0, 145],
        'y': [0.02, 0.02],
        'name': "Lower detection limit of assay",
        'line': {
            'color': 'rgb(100,100,100)',
            'dash': 'dashdot',
        },
    })
]

fig = go.FigureWidget(figure)

pio.write_image(fig, str(results_path / 'formate.svg'))
fig

FigureWidget({
    'data': [{'error_y': {'array': array([0, 0, 0, 0], dtype=int64), 'type': 'data', 'visible':…

## Scatter with most important means

In [72]:
axis_map = {
    "Concentration": {
        "index": 1,
        "position": "left"
    },
    "Optical Extinction": {
        "index": 2,
        "position": "right"
    }
}

fig = go.FigureWidget(
    plot_measurements(
        mean_data, 
        {
            ("EtOH", "Concentration"): "mean",
            ("Acetate", "Concentration"): "mean",
            ("OD 600", "Optical Extinction"): "mean"
        }, 
        axis_map
    )
)

pio.write_image(fig, str(results_path / 'growth-and-production.svg'))
fig

FigureWidget({
    'data': [{'error_y': {'array': array([       nan,        nan, 0.29096017, 0.05587874, 0.073…

## Correlation of Fructose and OD_600

In [75]:
axis_map = {
    "Concentration": {
        "index": 1,
        "position": "left"
    },
    "Optical Extinction": {
        "index": 2,
        "position": "right"
    }
}

fig = go.FigureWidget(
    plot_measurements(
        mean_data, 
        {
            ("Fructose", "Concentration"): "mean",
            ("OD 600", "Optical Extinction"): "mean"
        }, 
        axis_map
    )
)

pio.write_image(fig, str(results_path / 'od-and-fructose.svg'))
fig

FigureWidget({
    'data': [{'error_y': {'array': array([0.01100814, 0.01243729, 0.00296945, 0.00255934]),
   …

In [76]:
mean_data.to_csv(intermediate_path / "mean_data_CO.csv")